In [3]:
import pandas as pd
import math


In [4]:
!kaggle datasets download -d gauravmalik26/food-delivery-dataset

Dataset URL: https://www.kaggle.com/datasets/gauravmalik26/food-delivery-dataset
License(s): other
 51%|███████████████████▍                  | 1.00M/1.95M [00:00<00:00, 1.78MB/s]
100%|██████████████████████████████████████| 1.95M/1.95M [00:00<00:00, 3.03MB/s]


In [5]:
!unzip food-delivery-dataset.zip

Archive:  food-delivery-dataset.zip
  inflating: Sample_Submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [37]:
data = pd.read_csv('train.csv')

print(data.columns)

data_of_interest = data[["Restaurant_longitude", "Restaurant_latitude","Delivery_location_longitude", "Delivery_location_latitude","Road_traffic_density", "Vehicle_condition","Type_of_vehicle","Weatherconditions", "Time_taken(min)"]]

data_of_interest.loc[data_of_interest["Type_of_vehicle"] == "scooter"]

#data_of_interest

# t = data.query('Type_of_vehicle == "scooter"')
# t = data[data['Type_of_vehicle'].str.contains('scooter')]
# t = data[data['Type_of_vehicle'].str.strip() == 'scooter']
# t = data_of_interest[data_of_interest['Type_of_vehicle'].str.strip() == 'scooter']
t = data_of_interest['Time_taken(min)'].str.replace('(min)','').str.strip()
t

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)'],
      dtype='object')


0        24
1        33
2        26
3        21
4        30
         ..
45588    32
45589    36
45590    16
45591    26
45592    36
Name: Time_taken(min), Length: 45593, dtype: object

In [5]:
def calc_dist_with_haversine(coord1: object, coord2: object):

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 3)
    return round(km, 3)


calc_dist_with_haversine([-0.116773, 51.510357], [-77.009003, 38.889931])


5897.658

In [ ]:
def speed_by_distance_and_time(total_kilometers: int, total_time: int):
    return total_kilometers / total_time

#mangler fra restaurant og til destination?

def estimate_time_by_estimators(total_distance: calc_dist_with_haversine, weather: str, traffic: str, type_of_vechicle: str, vehicle_condition: str):
    scooter = data[data['Type_of_vehicle'].str.strip() == 'scooter']
    average_time = speed_by_distance_and_time(total_distance, data_of_interest['Time_taken(min)'].str.replace('(min)','').str.strip().sum)

    return ''